In [1]:
import csv
from IPython.display import clear_output

In [2]:
def parse_domain(dom_str):

    def sort_domains(domain_list):
        idx = sorted(range(len(domain_list)), key=lambda k: domain_list[k][1][0])
        return [domain_list[i] for i in idx]

    unsorted = [[it[0], list(map(int, it[1].split(':')))] for it in (dom.split(';') for dom in dom_str.split('|'))]

    return sort_domains(unsorted)

In [3]:
def check_gaps(sorted_doms, prot_len, intra_dom_thresh, terminal_thresh):

    if len(sorted_doms) == 0:
        return True
    if sorted_doms[0][1][0] - 1 > terminal_thresh:
        return True
    end = sorted_doms[0][1][1]
    if len(sorted_doms) > 1:
        for _, dom in sorted_doms[1:]:
            start = dom[0]
            if start - end - 1 > intra_dom_thresh:
                return True
            else:
                end = dom[1]
    if prot_len - end > terminal_thresh:
        return True
    return False

In [4]:
def domain_list_to_str(domain_list):
    dom_str = ''
    for dom in domain_list:
        dom_str += dom[0] + ';' + str(dom[1][0]) + ':' + str(dom[1][1]) + '|'
    return dom_str[:-1]

In [5]:
motifs = set()
with open('conserved_site.tsv') as f:
    rdr = csv.reader(f, delimiter='\t')
    next(rdr)
    for line in rdr:
        motifs.add(line[0])

In [7]:
with open('complete_dataset_no_consec.csv') as f:
    with open('gap_proteins.csv', 'w') as f_gap:
        with open('coverage_proteins_unprocessed.csv', 'w') as f_coverage:
            reader = csv.reader(f)
            for idx, line in enumerate(reader):
                annot_list = parse_domain(dom_str=line[1])
                sorted_doms = [dom for dom in annot_list if ((dom[0] not in motifs) and (not (dom[0]=='d' and (dom[1][1]-dom[1][0]<=40))))]
                if check_gaps(sorted_doms=sorted_doms, prot_len=int(line[2]), intra_dom_thresh=30, terminal_thresh=30):
                    f_gap.write(line[0]+','+domain_list_to_str(annot_list)+','+line[2]+'\n')
                else:
                    f_coverage.write(line[0]+','+domain_list_to_str(annot_list)+','+line[2]+'\n')
                if idx % 1000000 == 0:
                    clear_output()
                    print(f'Processing line: {idx}')

Processing line: 161000000


In [8]:
with open('coverage_proteins_unprocessed.csv') as f:
    with open('coverage_proteins.csv', 'w') as fOut:
        for idx, line in enumerate(f):
            dom_list = parse_domain(line.split(',')[1])
            dom_list_2 = []
            for dom in dom_list:
                if dom[0]=='d' and dom[1][1]-dom[1][0]<=40:
                    continue
                elif dom[0] in motifs:
                    dom_list_2.append(['m'+dom[0][2:], dom[1]])
                else:
                    dom_list_2.append(dom)
            fOut.write(line.split(',')[0]+','+domain_list_to_str(dom_list_2)+','+line[:-1].split(',')[2]+'\n')
            if idx % 1000000 == 0:
                clear_output()
                print()